In [1]:
import tensorflow as tf

In [ ]:
random_float = tf.random.uniform(shape=())

In [ ]:
print(random_float)

In [ ]:
random_float

In [ ]:
zero_vector = tf.zeros(shape=(2))

In [ ]:
zero_vector

In [ ]:
print(zero_vector)

In [ ]:
A = tf.constant([[1., 2.], [3., 4.]])

In [ ]:
B = tf.constant([[5., 6.], [7., 8.]])

In [ ]:
print(A.shape)

In [ ]:
print(A.dtype)

In [ ]:
print(A.numpy())

In [ ]:
zero_vector = tf.zeros(shape=(2), dtype=tf.int32)

In [ ]:
zero_vector

In [ ]:
C = tf.add(A, B)

In [ ]:
D = tf.matmul(A, B)

In [ ]:
print(C)

In [ ]:
print(D)

In [ ]:
x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    y = tf.square(x)
y_grad = tape.gradient(y, x)
print([y, y_grad])

In [ ]:
print((y, y_grad))

In [ ]:
X = tf.constant([[1., 2.], [3., 4.]])

In [ ]:
y = tf.constant([[1.], [2.]])

In [ ]:
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

In [ ]:
import numpy as np

In [ ]:
X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [ ]:
a, b = 0, 0

num_epoch = 10000
learning_rate = 1e-3
for e in range(num_epoch):
    y_pred = a * X + b
    grad_a, grad_b = (y_pred - y).dot(X), (y_pred - y).sum()
    a, b = a -learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

In [ ]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
for e in range(num_epoch):
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y))
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
    
print(a, b)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
    def call(self, input):
        return output

In [ ]:
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
    
    def call(self, input):
        output = self.dense(input)
        return output

model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)
        self.train_label = self.train_label.astype(np.int32)
        self.test_label = self.test_label.astype(np.int32)
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

In [4]:
import numpy as np
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [5]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.442612
batch 1: loss 2.298374
batch 2: loss 2.190372
batch 3: loss 2.087108
batch 4: loss 2.012150
batch 5: loss 1.880485
batch 6: loss 2.023318
batch 7: loss 1.902155
batch 8: loss 1.756096
batch 9: loss 1.766107
batch 10: loss 1.560382
batch 11: loss 1.661862
batch 12: loss 1.514317
batch 13: loss 1.747954
batch 14: loss 1.503443
batch 15: loss 1.490664
batch 16: loss 1.517150
batch 17: loss 1.497857
batch 18: loss 1.535744
batch 19: loss 1.273009
batch 20: loss 1.112091
batch 21: loss 1.262639
batch 22: loss 1.176915
batch 23: loss 0.993103
batch 24: loss 0.997267
batch 25: loss 1.135810
batch 26: loss 1.081021
batch 27: loss 1.004039
batch 28: loss 1.002007
batch 29: loss 0.949880
batch 30: loss 1.207240
batch 31: loss 1.094774
batch 32: loss 0.833957
batch 33: loss 0.986400
batch 34: loss 0.741791
batch 35: loss 0.980764
batch 36: loss 0.866698
batch 37: loss 0.668384
batch 38: loss 0.873301
batch 39: loss 0.890625
batch 40: loss 0.691412
batch 41: loss 0.780347
ba

In [6]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
print("number of data:", data_loader.num_test_data)
print("number of batch", num_batches)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    y_true = data_loader.test_label[start_index: end_index]
    print("y_true.shape:", y_true.shape)
    sparse_categorical_accuracy.update_state(y_true=y_true, y_pred=y_pred)
    
#     if batch_index == 1:
#         print("y_pred", y_pred.shape)
#         print(y_pred)
#         print("y_true", y_true.shape)
#         print(y_true)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

number of data: 10000
number of batch 200
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,)
y_true.shape: (50,

In [7]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output= tf.nn.softmax(x)
        return output

In [8]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    y_true = data_loader.test_label[start_index: end_index]
    sparse_categorical_accuracy.update_state(y_true=y_true, y_pred=y_pred)  
#     if batch_index == 1:
#         print("y_pred", y_pred.shape)
#         print(y_pred)
#         print("y_true", y_true.shape)
#         print(y_true)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

batch 0: loss 2.320215
batch 1: loss 2.308187
batch 2: loss 2.150623
batch 3: loss 1.996847
batch 4: loss 1.876315
batch 5: loss 1.781504
batch 6: loss 1.434969
batch 7: loss 1.337495
batch 8: loss 1.154074
batch 9: loss 1.064070
batch 10: loss 0.852002
batch 11: loss 0.632985
batch 12: loss 0.911466
batch 13: loss 0.867430
batch 14: loss 0.873742
batch 15: loss 1.149722
batch 16: loss 0.597406
batch 17: loss 0.443512
batch 18: loss 0.819830
batch 19: loss 0.818755
batch 20: loss 0.363359
batch 21: loss 0.486400
batch 22: loss 0.465859
batch 23: loss 0.567105
batch 24: loss 0.546432
batch 25: loss 0.330564
batch 26: loss 0.259285
batch 27: loss 0.375038
batch 28: loss 0.408953
batch 29: loss 0.322378
batch 30: loss 0.270779
batch 31: loss 0.232825
batch 32: loss 0.342714
batch 33: loss 0.422488
batch 34: loss 0.385214
batch 35: loss 0.467900
batch 36: loss 0.497945
batch 37: loss 0.213650
batch 38: loss 0.281959
batch 39: loss 0.369430
batch 40: loss 0.296839
batch 41: loss 0.341473
ba